# CSGY 6953 Deep Learning Final Project
In this project, we will implement a dual-encoder model for image search. In particular, our model will be fed a text query and will return several images that are related to the query. To do this task, our model will be trained so that it embeds both image and text data into the same space, and importantly, encodes relevant data to be close each other in the embedding space. This will be done by developing two encoders, one for image processing and the other for text encoding, and training them by a similarity-based loss function. <br>
<br>
In this notebook, I download the COCO image data to my Google drive, construct the dataset for training the dual-encoder.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import requests
import json
import cv2
import random
import tqdm
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
with open("./../nocaps_val_4500_captions.json", "r") as f:
    captions = json.load(f)

In [ ]:
# Downloading all the images to Google drive. It takes approx. 2 hours.
for image in tqdm.tqdm(captions["images"]):
    img = requests.get(image["coco_url"]).content
    with open("drive/My Drive/finalproj/data/images/"+str(image["id"]), "w") as handler:
        handler.write(img)

In [19]:
image_size = (299, 299)
images = []
for i in tqdm.tqdm(range(captions["images"][-1]["id"]+1)):
    details = captions["images"][i]
    id_ = details['id']
    image = cv2.imread("./../nocaps/images/"+str(details['file_name']))
    print(image.shape)
    images.append(cv2.resize(image, image_size))

  0%|                                        | 11/4500 [00:00<00:40, 109.52it/s]

(1024, 732, 3)
(768, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(683, 1024, 3)
(713, 1024, 3)
(1024, 768, 3)
(683, 1024, 3)
(683, 1024, 3)
(1024, 1013, 3)
(1024, 1024, 3)
(768, 1024, 3)
(866, 1024, 3)
(684, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(683, 1024, 3)
(768, 768, 3)
(1024, 1024, 3)
(1024, 1024, 3)
(866, 1024, 3)
(768, 1024, 3)
(683, 1024, 3)
(1024, 683, 3)


  1%|▍                                       | 51/4500 [00:00<00:35, 126.84it/s]

(683, 1024, 3)
(682, 1024, 3)
(681, 1024, 3)
(1024, 780, 3)
(768, 1024, 3)
(1024, 1024, 3)
(768, 1024, 3)
(681, 1024, 3)
(1024, 768, 3)
(768, 1024, 3)
(683, 1024, 3)
(683, 1024, 3)
(768, 1024, 3)
(685, 1024, 3)
(684, 1024, 3)
(768, 1024, 3)
(1024, 731, 3)
(640, 1024, 3)
(1024, 678, 3)
(1024, 685, 3)
(768, 1024, 3)
(1024, 683, 3)
(769, 1024, 3)
(1024, 683, 3)
(685, 1024, 3)
(1024, 663, 3)
(753, 1024, 3)


  1%|▌                                       | 64/4500 [00:00<00:36, 121.44it/s]

...

(768, 1024, 3)
(1024, 819, 3)
(703, 1024, 3)
(683, 1024, 3)
(683, 1024, 3)
(678, 1024, 3)
(577, 1024, 3)
(1024, 1024, 3)
(682, 1024, 3)
(1024, 1024, 3)
(1024, 769, 3)
(768, 1024, 3)
(768, 1024, 3)
(683, 1024, 3)
(768, 1024, 3)
(683, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(1024, 683, 3)
(1024, 681, 3)
(683, 1024, 3)
(1024, 768, 3)
(1024, 591, 3)
(768, 1024, 3)
(682, 1024, 3)
(1024, 1024, 3)
(768, 1024, 3)
(683, 1024, 3)
(768, 1024, 3)
(684, 1024, 3)
(683, 1024, 3)


100%|█████████████████████████████████████▊| 4478/4500 [00:30<00:00, 141.40it/s]

(683, 1024, 3)
(682, 1024, 3)
(768, 1024, 3)
(1024, 1024, 3)
(680, 1024, 3)
(781, 1024, 3)
(525, 1024, 3)
(758, 1024, 3)
(768, 1024, 3)
(682, 1024, 3)
(879, 1024, 3)
(1024, 684, 3)
(1024, 1024, 3)
(1024, 768, 3)
(639, 1024, 3)
(686, 1024, 3)
(768, 1024, 3)
(680, 1024, 3)
(1017, 1024, 3)
(683, 1024, 3)
(704, 1024, 3)
(1024, 768, 3)
(1024, 768, 3)
(575, 1024, 3)
(768, 1024, 3)


100%|██████████████████████████████████████| 4500/4500 [00:31<00:00, 144.63it/s]

(733, 1024, 3)
(678, 1024, 3)
(683, 1024, 3)
(768, 1024, 3)
(754, 1024, 3)
(819, 1024, 3)
(859, 1024, 3)
(683, 1024, 3)
(683, 1024, 3)
(574, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(1024, 769, 3)
(1024, 678, 3)
(1024, 680, 3)
(1024, 768, 3)
(683, 1024, 3)
(1024, 683, 3)
(1024, 685, 3)
(640, 1024, 3)
(768, 1024, 3)
(1024, 682, 3)


In [12]:
captions["images"][-1]

{'height': 1024,
 'file_name': 'fe7ed0dac010fdf0.jpg',
 'width': 682,
 'coco_url': 'https://s3.amazonaws.com/nocaps/val/fe7ed0dac010fdf0.jpg',
 'open_images_id': 'fe7ed0dac010fdf0',
 'id': 4499,
 'license': 0,
 'domain': 'near-domain',
 'date_captured': '2018-11-06 11:04:33'}

In [20]:
texts = []
for dic in tqdm.tqdm(captions["annotations"]):
    texts.append(dic["caption"])

100%|████████████████████████████████| 45000/45000 [00:00<00:00, 1650044.85it/s]


In [24]:
with open("./../dataset.pkl", "wb") as f:
    pickle.dump({"images": images, "texts": texts}, f)